In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)

In [2]:
synonyms = {
    'Здравоохранение/Медицина': ['лечение', 'медицинские услуги', 'здравотдел', 'здравпункт', 'мед. обслуживание', 'мин. здрав'],
    'Социальное обслуживание и защита': ['соцобслуживание', 'социальная защита', 'благотворительность', "льготы и пособие", 'социальная поддержка', 'социальная помощь'],
    'Дороги': ['автодороги', 'шоссе', 'трассы', 'проезжая часть', 'дорожная сеть'],
    'ЖКХ': ['коммунальные услуги', 'жилищное хозяйство', 'коммуналка', 'ЖКУ', 'жилищно-коммунальные работы'],
    'Благоустройство': ['озеленение', 'ландшафтный дизайн', 'уличная обстановка', 'уборка территории', 'улучшение территории'],
    'Мусор/Свалки/ТКО': ['отходы', 'утилизация мусора', 'захоронение отходов', 'переработка мусора', 'уборка отходов'],
    'Общественный транспорт': ['пассажирские перевозки', 'общественный транспорт', 'городской транспорт', 'муниципальный транспорт', 'автобусы и троллейбусы', 'оставновки и транспорт'],
    'Коронавирус': ['COVID-19', 'пандемия', 'вирусная инфекция', 'корона', 'эпидемия коронавируса'],
    'Образование': ['педагогика', 'учебный процесс', 'школьное обучение', 'образовательные услуги', 'учебные заведения'],
    'Безопасность': ['охрана порядка', 'профилактика преступлений', 'защита', 'безопасное пространство', 'недопущение нарушений'],
    'Связь и телевидение': ['телекоммуникации', 'масс-медиа', 'информационные сервисы', 'радиовещание', 'информационная индустрия'],
    'Мобилизация': ['призыв', 'военный набор', 'резервисты', 'подготовка к обороне', 'мобилизационная подготовка', 'СВО', 'военная операция'],
    'Физическая культура и спорт': ['спортивная деятельность', 'физкультура', 'спортивные мероприятия', 'физическое воспитание', 'спорт'],
    'Строительство и архитектура': ['застройка', 'архитектурное планирование', 'стройиндустрия', 'градостроительство', 'строительные работы'],
    'Газ и топливо': ['газоснабжение', 'топливная индустрия', 'нефтегаз', 'энергоресурсы', 'газовая промышленность'],
    'Спецпроекты': ['особые программы', 'специальные инициативы', 'эксклюзивные проекты', 'инновационные проекты', 'спецработы'],
    'Культура': ['искусство', 'духовное наследие', 'культурное развитие', 'творчество', 'культурная жизнь'],
    'Электроснабжение': ['энергоснабжение', 'электроэнергия', 'подача тока', 'электросеть', 'энергетика'],
    'Экономика и бизнес': ['экономическая деятельность', 'бизнес-сектор', 'финансы и торговля', 'предпринимательство', 'торгово-экономический комплекс'],
    'Экология': ['охрана окружающей среды', 'природопользование', 'экосистема', 'природозащита', 'экологическая безопасность'],
    'Роспотребнадзор': ['защита прав потребителей', 'санитарный надзор', 'надзор за услугами', 'государственный контроль', 'контроль качества'],
    'Памятники и объекты культурного наследия': ['исторические объекты', 'культурные ценности', 'заповедники', 'архитектурное наследие', 'музей под открытым небом'],
    'Государственная собственность': ['госимущество', 'государственные активы', 'федеральная собственность', 'муниципальное имущество', 'державные резервы'],
    'Торговля': ['коммерция', 'продажи', 'рыночная деятельность', 'внешняя и внутренняя торговля', 'торговый бизнес'],
    'МФЦ "Мои документы"': ['центры обслуживания', 'госуслуги', 'административные сервисы', 'общественные сервисы', 'документационное обслуживание'],
    # 'Погребение и похоронное дело' - для этого термина трудно подобрать синонимы, которые точно передавали бы суть деятельности без контекстной потери.
    'Погребение и похоронное дело': ['ритуальные услуги', 'похоронная служба', 'захоронение', 'ритуальная служба', 'похороны']
}

In [3]:
def get_dicts(results):
    dicts_ = []
    for result in results:
        label_score_pairs = list(zip(result['labels'], result['scores']))
        
        # Use a dictionary comprehension to convert pairs into a dictionary
        label_score_dict = {label: score for label, score in label_score_pairs}
        dicts_.append(label_score_dict)
    return dicts_

In [4]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

In [5]:
data = pd.read_csv("data/train.csv")
texts = data["Текст инцидента"].to_list()
labels = list(data["Группа тем"].value_counts().keys())
expanded_labels = ["{} ({})".format(label, ', '.join(synonyms[label])) for label in labels]

In [6]:
MODELS = [("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", "mdbrt_xnl"), ('cointegrated/rubert-base-cased-nli-threeway', "rbrt_3")]

In [7]:
for model_info in MODELS:
    hugging_name, name = model_info
    tokenizer = AutoTokenizer.from_pretrained(hugging_name)
    model = AutoModelForSequenceClassification.from_pretrained(hugging_name)

    if torch.cuda.is_available():
        model.cuda()

    labels_ = [f"{name}_{label}" for label in labels]
    zero_shot = []

    for text in tqdm(texts):
        zero_shot.append(dict(zip(labels_, predict_zero_shot(text, expanded_labels, model, tokenizer, label='entailment', normalize=True))))
    
    texts_df = pd.DataFrame(zero_shot)
    df_res = pd.concat([data, texts_df], axis=1)

    df_res.to_csv(f"data/{name}_train.csv", index=False)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 22530/22530 [07:37<00:00, 49.25it/s]


In [8]:
df_res

,Исполнитель,Группа тем,Текст инцидента,Тема,rbrt_3_Здравоохранение/Медицина,rbrt_3_Социальное обслуживание и защита,rbrt_3_Дороги,rbrt_3_ЖКХ,rbrt_3_Благоустройство,rbrt_3_Мусор/Свалки/ТКО,...,rbrt_3_Культура,rbrt_3_Электроснабжение,rbrt_3_Экономика и бизнес,rbrt_3_Экология,rbrt_3_Роспотребнадзор,rbrt_3_Памятники и объекты культурного наследия,rbrt_3_Государственная собственность,rbrt_3_Торговля,"rbrt_3_МФЦ ""Мои документы""",rbrt_3_Погребение и похоронное дело
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,0.048705,0.052714,0.161883,0.022284,0.047180,0.007163,...,0.022664,0.053838,0.005230,0.015606,0.007825,0.026431,0.010285,0.013577,0.005685,0.025486
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи,0.084328,0.050574,0.169336,0.024782,0.006744,0.025475,...,0.014172,0.081784,0.034773,0.010710,0.009292,0.019509,0.031808,0.046866,0.002858,0.005327
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи,0.101843,0.094698,0.088189,0.021134,0.008967,0.010250,...,0.064134,0.078602,0.042157,0.018911,0.006296,0.019397,0.017114,0.068213,0.011773,0.009154
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,0.015793,0.023579,0.152098,0.064346,0.005393,0.007394,...,0.006972,0.113489,0.020463,0.010554,0.006124,0.014456,0.017669,0.083023,0.034779,0.001617
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,0.140910,0.090894,0.062690,0.008866,0.008660,0.007999,...,0.030991,0.073589,0.017472,0.008722,0.020612,0.020572,0.026373,0.031598,0.011618,0.021792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22525,Министерство социального развития ПК,Социальное обслуживание и защита,", а если ещё не погасили ипотеку, но площадь ...",Улучшение жилищных условий,0.036109,0.064556,0.092026,0.022015,0.011203,0.013414,...,0.014991,0.082175,0.065991,0.016040,0.001908,0.014138,0.012725,0.155337,0.007212,0.009298
22526,Губахинский городской округ,ЖКХ,Город Гремячинск ситуация с теплом на улице Л...,Ненадлежащее качество или отсутствие отопления,0.026174,0.021576,0.215687,0.104945,0.045970,0.012817,...,0.014203,0.108728,0.006279,0.014423,0.002831,0.030243,0.008356,0.049310,0.003079,0.001506
22527,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу,0.102063,0.161751,0.116437,0.003653,0.006009,0.001943,...,0.046069,0.032656,0.016048,0.015095,0.010671,0.015194,0.007638,0.018427,0.009114,0.007993
22528,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов. Воо...,Благоустройство придомовых территорий,0.013562,0.042138,0.134871,0.083667,0.055239,0.030713,...,0.014725,0.030569,0.010481,0.038122,0.009268,0.054543,0.023219,0.011894,0.005213,0.001070


In [ ]:
zero_shot = []

for text in tqdm(texts):
    zero_shot.append(dict(zip(labels_, predict_zero_shot(text, expanded_labels, model, tokenizer, label='entailment', normalize=True))))

  1%|          | 224/22530 [00:10<18:12, 20.42it/s]

In [53]:
texts_df = pd.DataFrame(zero_shot)

In [57]:
df_res = pd.concat([data, texts_df], axis=1)

In [59]:
df_res.head(3)

,Исполнитель,Группа тем,Текст инцидента,Тема,Здравоохранение/Медицина,Социальное обслуживание и защита,Дороги,ЖКХ,Благоустройство,Мусор/Свалки/ТКО,...,Культура,Электроснабжение,Экономика и бизнес,Экология,Роспотребнадзор,Памятники и объекты культурного наследия,Государственная собственность,Торговля,"МФЦ ""Мои документы""",Погребение и похоронное дело
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,0.032651,0.038203,0.095653,0.033030,0.169776,0.016031,...,0.016146,0.027807,0.033891,0.030938,0.027881,0.020490,0.017793,0.029376,0.031591,0.041036
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи,0.049973,0.056974,0.057099,0.050486,0.051638,0.046177,...,0.016504,0.044984,0.055208,0.045229,0.001087,0.004695,0.042820,0.055389,0.005975,0.051682
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи,0.045039,0.047977,0.047473,0.048638,0.049333,0.047752,...,0.037862,0.044122,0.047773,0.047779,0.015893,0.002648,0.024886,0.047282,0.021684,0.049017


In [38]:
data[:2]

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи


In [39]:
data[:2]["Текст инцидента"].to_list()

[' Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.',
 ' Пермь г, 79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется']

In [40]:
# Вывод топ-3 из каждого словаря
for text, (i, dictionary) in zip(data[:5].iterrows(), enumerate(zero_shot, 1)):
    print(text[1]["Группа тем"], text[1]["Текст инцидента"], sep='\n')

Благоустройство
 Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.

Топ 3 для словаря 1:
Благоустройство: 0.16977643966674805
Дороги: 0.09565259516239166
Безопасность: 0.07006222754716873


----------
Социальное обслуживание и защита
 Пермь г, 79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется

Топ 3 для словаря 2:
Общественный транспорт: 0.05749836936593056
Дороги: 0.057098791003227234
Социальное обслуживание и защита: 0.056974463164806366


----------


In [ ]:
    top_3 = sorted(dictionary.items(), key=lambda item: item[1], reverse=True)[:3]
    print(f"\nТоп 3 для словаря {i}:")
    
    for term, value in top_3:
        print(f"{term}: {value}")
    print("\n")  # Добавим пустую строку для разделения вывода для разных словарей
    print('-'*10)

In [12]:
zero_shot

[{'Здравоохранение/Медицина': 0.07243349,
  'Социальное обслуживание и защита': 0.057399888,
  'Дороги': 0.1599418,
  'ЖКХ': 0.022017324,
  'Благоустройство': 0.046614457,
  'Мусор/Свалки/ТКО': 0.0070770737,
  'Общественный транспорт': 0.12572664,
  'Коронавирус': 0.03670317,
  'Образование': 0.008089678,
  'Безопасность': 0.13498378,
  'Связь и телевидение': 0.024683768,
  'Мобилизация': 0.0012552217,
  'Физическая культура и спорт': 0.07612989,
  'Строительство и архитектура': 0.010305518,
  'Газ и топливо': 0.0149787385,
  'Спецпроекты': 0.017268587,
  'Культура': 0.02239249,
  'Электроснабжение': 0.053192865,
  'Экономика и бизнес': 0.0051672845,
  'Экология': 0.015419055,
  'Роспотребнадзор': 0.007731632,
  'Памятники и объекты культурного наследия': 0.0261142,
  'Государственная собственность': 0.010161442,
  'Торговля': 0.01341474,
  'МФЦ "Мои документы"': 0.0056168595,
  'Погребение и похоронное дело': 0.025180386},
 {'Здравоохранение/Медицина': 0.12884903,
  'Социальное обслуж

In [17]:
data[:3]

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи


In [15]:
list(data.iloc[:3]["Текст инцидента"])

[' Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.',
 ' Пермь г, 79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется',
 ' Добрый день . Скажите пожалуйста если подовала на пособие с 3 до 7 2 декабря , когда можно повторно подать ? вроде за 30 дней можно']

In [20]:
zero_shot[2]

{'Здравоохранение/Медицина': 0.0369621,
 'Социальное обслуживание и защита': 0.08136316,
 'Дороги': 0.060928594,
 'ЖКХ': 0.0012773033,
 'Благоустройство': 0.013771566,
 'Мусор/Свалки/ТКО': 0.03410873,
 'Общественный транспорт': 0.00032323206,
 'Коронавирус': 0.00041167648,
 'Образование': 0.00071735826,
 'Безопасность': 0.00058839784,
 'Связь и телевидение': 0.0017441931,
 'Мобилизация': 0.24488316,
 'Физическая культура и спорт': 0.0033227406,
 'Строительство и архитектура': 0.0052733435,
 'Газ и топливо': 0.008030815,
 'Спецпроекты': 0.24313308,
 'Культура': 0.0008101569,
 'Электроснабжение': 0.0029854586,
 'Экономика и бизнес': 0.001905556,
 'Экология': 0.00063697476,
 'Роспотребнадзор': 0.00092287146,
 'Памятники и объекты культурного наследия': 0.001229014,
 'Государственная собственность': 0.13127658,
 'Торговля': 0.09587384,
 'МФЦ "Мои документы"': 0.0022251587,
 'Погребение и похоронное дело': 0.025294904}

In [5]:
", ".join(labels)

'Здравоохранение/Медицина, Социальное обслуживание и защита, Дороги, ЖКХ, Благоустройство, Мусор/Свалки/ТКО, Общественный транспорт, Коронавирус, Образование, Безопасность, Связь и телевидение, Мобилизация, Физическая культура и спорт, Строительство и архитектура, Газ и топливо, Спецпроекты, Культура, Электроснабжение, Экономика и бизнес, Экология, Роспотребнадзор, Памятники и объекты культурного наследия, Государственная собственность, Торговля, МФЦ "Мои документы", Погребение и похоронное дело'

In [ ]:
show_data = []
for batch in tqdm(batches):
    output = classifier(batch, labels, multi_label=False)
    result = get_dicts(output)
    show_data.extend(result)

  0%|          | 5/1409 [01:30<6:52:02, 17.61s/it]

In [7]:
print(data.iloc[0]["Текст инцидента"])

 Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.


In [16]:
data["Группа тем"]

0                         Благоустройство
1        Социальное обслуживание и защита
2        Социальное обслуживание и защита
3                  Общественный транспорт
4                Здравоохранение/Медицина
                       ...               
22525    Социальное обслуживание и защита
22526                                 ЖКХ
22527            Здравоохранение/Медицина
22528                     Благоустройство
22529    Социальное обслуживание и защита
Name: Группа тем, Length: 22530, dtype: object

In [8]:
show_data

[{'Дороги': 0.4742022156715393,
  'Благоустройство': 0.3620149791240692,
  'Безопасность': 0.11064452677965164,
  'Мобилизация': 0.02219863422214985,
  'Государственная собственность': 0.0056534986943006516,
  'Общественный транспорт': 0.005497244652360678,
  'ЖКХ': 0.0028140961658209562,
  'Электроснабжение': 0.0025275512598454952,
  'Социальное обслуживание и защита': 0.002205977449193597,
  'Здравоохранение/Медицина': 0.001733725075609982,
  'Погребение и похоронное дело': 0.001085619325749576,
  'Газ и топливо': 0.0010614615166559815,
  'Спецпроекты': 0.0010354158002883196,
  'Строительство и архитектура': 0.000891435076482594,
  'Роспотребнадзор': 0.0007872450514696538,
  'Экономика и бизнес': 0.0006987701053731143,
  'Памятники и объекты культурного наследия': 0.0006359101971611381,
  'Культура': 0.0006162405479699373,
  'Связь и телевидение': 0.0005868949228897691,
  'Торговля': 0.0005605738260783255,
  'Мусор/Свалки/ТКО': 0.00046806089812889695,
  'Экология': 0.0004574385529849

In [5]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1", device=0)

In [6]:
data.iloc[0]["Текст инцидента"]

' Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.'

In [7]:
sequence_to_classify = data.iloc[0]["Текст инцидента"]
candidate_labels = labels
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)


In [8]:
get_dicts([output])

[{'Благоустройство': 0.0673627108335495,
  'Коронавирус': 0.05220114067196846,
  'Торговля': 0.049726326018571854,
  'Дороги': 0.048627957701683044,
  'Мобилизация': 0.04839465767145157,
  'Мусор/Свалки/ТКО': 0.045927710831165314,
  'Безопасность': 0.04461669921875,
  'Культура': 0.04308779165148735,
  'Здравоохранение/Медицина': 0.042335279285907745,
  'Связь и телевидение': 0.041586291044950485,
  'Экология': 0.03877926245331764,
  'Спецпроекты': 0.038727931678295135,
  'Роспотребнадзор': 0.03718283027410507,
  'Образование': 0.03593577444553375,
  'ЖКХ': 0.03519650921225548,
  'Социальное обслуживание и защита': 0.03485891595482826,
  'Газ и топливо': 0.03411963954567909,
  'Электроснабжение': 0.033277422189712524,
  'Физическая культура и спорт': 0.0329168327152729,
  'Строительство и архитектура': 0.03284507244825363,
  'Экономика и бизнес': 0.03197817504405975,
  'Государственная собственность': 0.03111763671040535,
  'Общественный транспорт': 0.028752202168107033,
  'Погребение 

In [ ]:
get

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

classes = labels
res = predict_zero_shot(data.iloc[0]["Текст инцидента"], classes, model, tokenizer)

In [9]:
dict(zip(labels, res))

{'Здравоохранение/Медицина': 0.00336944,
 'Социальное обслуживание и защита': 0.54626316,
 'Дороги': 0.0043771574,
 'ЖКХ': 0.0046370993,
 'Благоустройство': 0.005680582,
 'Мусор/Свалки/ТКО': 0.0020164954,
 'Общественный транспорт': 0.011570638,
 'Коронавирус': 0.0018653381,
 'Образование': 0.002354904,
 'Безопасность': 0.044092167,
 'Связь и телевидение': 0.0062241484,
 'Мобилизация': 0.0150900455,
 'Физическая культура и спорт': 0.0030005423,
 'Строительство и архитектура': 0.0074633164,
 'Газ и топливо': 0.013458099,
 'Спецпроекты': 0.039699323,
 'Культура': 0.0027449788,
 'Электроснабжение': 0.18732905,
 'Экономика и бизнес': 0.005688766,
 'Экология': 0.0027789448,
 'Роспотребнадзор': 0.00961375,
 'Памятники и объекты культурного наследия': 0.0034370269,
 'Государственная собственность': 0.06210282,
 'Торговля': 0.0057876017,
 'МФЦ "Мои документы"': 0.0028142086,
 'Погребение и похоронное дело': 0.006540402}

In [10]:
labels

['Здравоохранение/Медицина',
 'Социальное обслуживание и защита',
 'Дороги',
 'ЖКХ',
 'Благоустройство',
 'Мусор/Свалки/ТКО',
 'Общественный транспорт',
 'Коронавирус',
 'Образование',
 'Безопасность',
 'Связь и телевидение',
 'Мобилизация',
 'Физическая культура и спорт',
 'Строительство и архитектура',
 'Газ и топливо',
 'Спецпроекты',
 'Культура',
 'Электроснабжение',
 'Экономика и бизнес',
 'Экология',
 'Роспотребнадзор',
 'Памятники и объекты культурного наследия',
 'Государственная собственность',
 'Торговля',
 'МФЦ "Мои документы"',
 'Погребение и похоронное дело']